In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import os
# For Analysis
import ipywidgets as widgets
from IPython.display import display, clear_output

# Aquiring Data

This will aquire the data from Drive once fully developed. Currently getting it from the data folder that is being ignored in Github. Right now, we are only getting it from folders Hits and Query, not Assembly.

In [2]:
#This will have to change after we set it up to pull from Drive
datapath =  os.getcwd() + "/data/"

In [3]:
# Getting our dataframe. Currently hardcoded with the name of the csv.
query = pd.read_csv(datapath + "query.csv")
hits = pd.read_csv(datapath + "hits.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/evandvance/bioinformatics/workflows/analysis-tools/data/query.csv'

# Cleaning Data

This section cleans the data and also shows exactly what was dropped/manipulated.

In [ ]:
# These functions are for later exception raising/preliminary analysis of the actual csv structure.
# They are mainly for cleaning the data, analysis functions will be later/in a different block.
def compare_columns_sets(df1,df2):
    set1_c, set2_c = set(df1.columns), set(df2.columns)
    # "a - b" removes the items in a that it shares with b. (just look up set theory)
    df1_dif = set1_c - set2_c
    df2_dif = set2_c - set1_c
    return df1_dif, df2_dif

def drop_nan_columns(df):
    dropped_df = df.dropna(axis=1,how='all')
    return dropped_df

def unusual_row_mask(df, col, threshold=0.5):
    """Return a Boolean mask for rows where the column has values,
    but the column is mostly NaN based on the threshold."""
    if df[col].isna().mean() >= threshold:
        return df[col].notna()
    return pd.Series([False] * len(df), index=df.index)

def get_peculiar_columns(df,threshold=0.5):
    return (df.isnull().sum() / df.shape[0])[df.isnull().sum() / df.shape[0] > threshold]

def rows_for_peculiar_columns(df,threshold=0.5):
    masks = {}
    p_series = get_peculiar_columns(df,threshold)
    for p_col in p_series.index:
        mask = unusual_row_mask(df, p_col, threshold)
        if mask.any():  # Only keep masks that select at least one row. A dictionary
            masks[p_col] = mask
    return masks

def split_query_name(row,splitting_column='Name'):
    name = row[splitting_column]
    name = name.split()
    for item in name:
        if "=" in item:
            item = item.split("=")
            if item[1] == '' or item[1] == ' ' or item[1] == []:
                item[1] = np.nan
            row[item[0]] = item[1]
    return row

def compare_columns_rowwise(df, col1, col2):
    """
    Compare two columns in a DataFrame row-wise.
    Returns:
        -1 if all values are the same in every row,
        otherwise, a tuple of row indices where values differ.
    """
    mask = df[col1] != df[col2]
    diff_indices = tuple(df.index[mask])
    return -1 if not diff_indices else diff_indices

def combine_col1_into_col2(df, col1, col2):
    """
    Combine values from col1 into col2 if col2 is NaN.
    If col2 is not NaN, it will keep the original value.
    """
    df.loc[:,col2] = df.loc[:,col1]
    return df.drop(columns=[col1], axis=1)  # Drops col1 after combining


### Immediate Cleanup

Some of the data has all NaN values for certain columns, so they need to be cleaned up. Also, some qualities of the unclean data are shown.

In [ ]:
print(len(query.columns), "columns in uncleaned query")
print(len(hits.columns), "columns in uncleaned hits")

31 columns in uncleaned query
38 columns in uncleaned hits


In [ ]:
query.columns

Index(['Name', '# Source Sequences', '% GC', '% HQ', '% Identical Sites',
       '% LQ', '% MQ', '% Pairwise Identity', 'Ambiguities', 'At least Q20',
       'At least Q30', 'At least Q40', 'Bin', 'Created', 'Created Date',
       'Description', 'Failed Binning Fields', 'Free end gaps',
       'Mean Coverage', 'Modified', 'Molecular Weight (kDa)', 'Molecule Type',
       'Post-Trim Length', 'Rough Temperature (°C)', 'Sample', 'Sequence',
       'Sequence Length', 'Sequence List Name', 'Size', 'Topology', 'URN'],
      dtype='object')

In [ ]:
hits.columns

Index(['Name', '# Nucleotides', '# Sequences', '% GC', '% Identical Sites',
       '% Pairwise Identity', 'Accession', 'Bit-Score', 'Created',
       'Created Date', 'Database', 'Description', 'E Value', 'Free end gaps',
       'Grade', 'Hit end', 'Hit start', 'Max Sequence Length', 'Mean Coverage',
       'Min Sequence Length', 'Modified', 'Molecular Weight (kDa)',
       'Molecule Type', 'Organism', 'Original Query Frame', 'Query',
       'Query Id', 'Query coverage', 'Query end', 'Query start',
       'Ref Seq Index', 'Ref Seq Length', 'Ref Seq Name',
       'Rough Temperature (°C)', 'Sequence', 'Sequence Length', 'Topology',
       'URN'],
      dtype='object')

In [ ]:
# Drops missing columns and combines same columns.
dropped_query = drop_nan_columns(query)
dropped_hits = drop_nan_columns(hits)
# Error check. Make Better Later

print("For dropped query", compare_columns_sets(query,dropped_query))

print("For dropped hits", compare_columns_sets(hits,dropped_hits))

if compare_columns_rowwise(dropped_query, 'Created Date', 'Created') != -1:
     raise ValueError(compare_columns_rowwise(dropped_query, 'Created Date', 'Created'), "are the rows where Created Date and Created differ.")

if compare_columns_rowwise(dropped_hits, 'Created Date', 'Created') != -1:
    raise ValueError(compare_columns_rowwise(dropped_hits, 'Created Date', 'Created'), "are the rows where Created Date and Created differ.")

# Combines Created Date into Created
dropped_query = combine_col1_into_col2(dropped_query, 'Created Date', 'Created')
dropped_hits = combine_col1_into_col2(dropped_hits, 'Created Date', 'Created')
print("Merged Created Date into Created in both query and hits.")

For dropped query ({'Sequence List Name'}, set())
For dropped hits ({'Query Id', 'Ref Seq Name'}, set())
Merged Created Date into Created in both query and hits.


### Metadata Cleanup

The exportation from Geneious Prime has extra data in Name and Query from the sequences in the folders Query and Hits respectively. These will enable us to partially link them together later in analysis.

This next block shows exactly what we have to split up in our data, as Geneious prime put more data inside certain cells than others.

In [ ]:
# Shows the peculiar columns in the query and hits.
print("There's metadata in certain cells.")
# Arbitrary row chosen to demonstrate the metadata.
query_val = dropped_hits.loc[10,["Query"]]
pd.set_option('display.max_colwidth', None)
print("From the Query column in hits:\n", query_val)
name_val = dropped_query.loc[20,["Name"]]
print("\nFrom the Name column in query:\n", name_val)

There's metadata in certain cells.
From the Query column in hits:
 Query    ace289cb-a9d5-470a-9e5f-86a43409224e runid=d0b33ea7460a391678012986097c20ea1c294534 ch=328 start_time=2025-02-25T14:51:59.325974-06:00 flow_cell_id=FBA87864 protocol_group_id=B3_T9_Seq_Run_25_02_25 sample_id= barcode=barcode13 barcode_alias=barcode13 parent_read_id=ace289cb-a9d5-470a-9e5f-86a43409224e basecall_model_version_id=dna_r10.4.1_e8.2_400bps_hac@v4.3.0
Name: 10, dtype: object

From the Name column in query:
 Name    1a42e9ad-2006-4a0d-8c47-45e11a2a0354 runid=d0b33ea7460a391678012986097c20ea1c294534 ch=358 start_time=2025-02-26T05:02:08.325974-06:00 flow_cell_id=FBA87864 protocol_group_id=B3_T9_Seq_Run_25_02_25 sample_id= barcode=barcode13 barcode_alias=barcode13 parent_read_id=1a42e9ad-2006-4a0d-8c47-45e11a2a0354 basecall_model_version_id=dna_r10.4.1_e8.2_400bps_hac@v4.3.0
Name: 20, dtype: object


Yet, for our queries that were contigs, the "Name" does not have the extra data (as it is just named "Contig #", so we lose a lot of extra data). We need to split the query dataframe.

In [ ]:
# Splits the query table into the contigs and noncontigs. + a TODO
# TODO May need to do equals() to see if each column really does correspond to a contig.
query_masks = rows_for_peculiar_columns(dropped_query)
print("These keys are what contigs but not regular sequences have:\n", query_masks.keys())
contig_query = dropped_query[query_masks['# Source Sequences']]
noncontig_query = drop_nan_columns(dropped_query[~query_masks['# Source Sequences']])

These keys are what contigs but not regular sequences have:
 dict_keys(['# Source Sequences', '% Identical Sites', '% Pairwise Identity', 'Description', 'Free end gaps', 'Mean Coverage', 'Sample'])


Some of the extra data, doesn't actually have extra data, so that also needs to be cleaned up.

In [ ]:
# Splits our data up and gets the weird cell split up. Also drops sample_id as there is none.
clean_contig_query = contig_query.apply(split_query_name, axis=1) # Nothing happens as Name is just Contig #.
clean_noncontig_query = drop_nan_columns(noncontig_query.apply(split_query_name, axis=1))
clean_hits = drop_nan_columns(dropped_hits.apply(split_query_name, axis=1, splitting_column="Query"))

In [ ]:
# Shows unique columns in contigs and non-contigs queries.
in_contig, not_in_contig = compare_columns_sets(clean_contig_query, clean_noncontig_query)
print("This is what is uniquely inside contigs:\n", in_contig, "\nThis is what is uniquely in non-contigs:\n", not_in_contig)

This is what is uniquely inside contigs:
 {'Description', '% Identical Sites', '# Source Sequences', 'Free end gaps', 'Mean Coverage', '% Pairwise Identity', 'Sample'} 
This is what is uniquely in non-contigs:
 {'parent_read_id', 'barcode_alias', 'basecall_model_version_id', 'runid', 'start_time', 'flow_cell_id', 'ch', 'protocol_group_id', 'barcode'}


In [ ]:
# Combine 'barcode' and 'barcode_alias' into 'barcode' 
clean_noncontig_query = combine_col1_into_col2(clean_noncontig_query, 'barcode_alias', 'barcode')
clean_hits = combine_col1_into_col2(clean_hits, 'barcode_alias', 'barcode')

In [ ]:
# Cleaning clean_hits where they are contigs.
# This is where we will add the barcode and protocol_group_id.
# We will also fill in the parent_read_id with the Query column.

clean_hits[['barcode', 'protocol_group_id']] = clean_hits[['barcode', 'protocol_group_id']].bfill().ffill()
clean_hits['parent_read_id'] = clean_hits['parent_read_id'].fillna(clean_hits['Query'])

Final look at what the dataframe looks like along with final bug tests before analysis.

In [ ]:
# Columns in c_hits
c_hits_val = clean_hits.loc[1, :]
pd.set_option('display.max_colwidth', None)
print("From a row in c_hits:\n", c_hits_val)

From a row in c_hits:
 # Nucleotides                                                                                                                                                                                                                                                                                                                                                                                         58
# Sequences                                                                                                                                                                                                                                                                                                                                                                                            2
% GC                                                                                                                                                                                           

In [ ]:
# Columns in c_contig_query
c_contig_val = clean_contig_query.iloc[3,:]
pd.set_option('display.max_colwidth', None)
print("Columns (+ examples) in c_contig_query:\n")
print("From a row in c_hits:\n", c_contig_val)

Columns (+ examples) in c_contig_query:

From a row in c_hits:
 Name                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             Contig 4
# Source Sequences                                                                                                                                                                                                                                                                                                            

In [ ]:
## TODO, make tests of data connections.
#assert all(hits.columns == query.columns), "Columns in hits and query do not match"
#assert all(hits.columns == query.columns), "Columns in hits and query do not match"

In [ ]:
## TODO, analysis of filtered workflow vs unfiltered workflow.
#filtered_hits = pd.read_csv(datapath + "barcode13-filtered-hits.csv")
#filtered_hits = filtered_hits.apply(split_query_name, axis=1)
#filtered_hits.columns
#hits[hits["parent_read_id"] != filtered_hits["parent_read_id"]]

# Analysis

The data has been cleaned up. Now it is time to use that to see the details about the run

In [ ]:
## Useful columns in clean_hits (so we can change later):
USEFUL_COLS = [
    'Name',
    'parent_read_id',
    'Accession',
    'Grade',
    'E Value',
    'Bit-Score',
    '% Identical Sites',
    '% Pairwise Identity',
    'barcode',
    'Sequence Length',
]
organism_percent = clean_hits['Organism'].value_counts(normalize=True) * 100

In [ ]:
# Widgets for analysis
# Look at https://ipywidgets.readthedocs.io/en/7.7.1/examples/Widget%20Basics.html for more info on widgets.
## Dropdown for organism selection
organism_dropdown = widgets.Dropdown(
    options=list(organism_percent.index),
    value=organism_percent.index[0],
    description='Organism:',
    disabled=False,
    continuous_update=False
)
## IntText for number of results to show
num_results_input = widgets.BoundedIntText(
    value=3,
    min=1,
    max=len(clean_hits),
    step=1,
    description='Top Grade #:',
    disabled=False,
    continuous_update=False
)
## Checkbox for ascending grade order
ascending_input = widgets.Checkbox(
    value=False,
    description='Ascending Grade Order',
    disabled=False,
    continuous_update=False
)


In [ ]:
# Functions for Analysis
## Output for showing results based on Grade from Organism
output_organism = widgets.Output(layout={'border': '1px solid black'})
def show_top_hits_dropdown(organism_name, n_results, ascending=False):
    filtered = clean_hits[clean_hits['Organism'] == organism_name]
    try:
        filtered = filtered.copy()
        filtered['Grade_numeric'] = filtered['Grade'].str.rstrip('%').astype(float)
        filtered = filtered.sort_values('Grade_numeric', ascending=ascending)
    except Exception:
        filtered = filtered.sort_values('Grade', ascending=ascending)
    display(filtered.loc[:,USEFUL_COLS].head(n_results))

# Updates output based on dropdown change
def on_dropdown_change(change):
    with output_organism:
        clear_output()
        show_top_hits_dropdown(organism_dropdown.value, num_results_input.value, ascending_input.value)
# Update the output when the num value changes
def on_num_results_change(change):
    with output_organism:
        clear_output()
        show_top_hits_dropdown(organism_dropdown.value, num_results_input.value, ascending_input.value)
# Update the output when the ascending value changes
def on_ascending_input_change(change):
    with output_organism:
        clear_output()
        show_top_hits_dropdown(organism_dropdown.value, num_results_input.value, ascending_input.value)

### Number of Unique Organisms Found

In [ ]:
# Initial display of top hits for the most common organism
for organism, percent in organism_percent.items():
    print(f"{organism} - {percent:.2f} % - {(clean_hits['Organism'] == organism).sum()} hits")

Klebsiella pneumoniae - 92.93 % - 684 hits
Escherichia coli - 3.26 % - 24 hits
Aspergillus citrinoterreus - 0.95 % - 7 hits
Escherichia phage - 0.41 % - 3 hits
Acinetobacter baumannii - 0.27 % - 2 hits
MAG: Pyrinomonadaceae - 0.27 % - 2 hits
Enterococcus faecium - 0.14 % - 1 hits
MAG: Burkholderiales - 0.14 % - 1 hits
Aquabacterium sp. - 0.14 % - 1 hits
Agrobacterium pusense - 0.14 % - 1 hits
Marinomonas arctica - 0.14 % - 1 hits
Candidatus Viadribacter - 0.14 % - 1 hits
Kaustia mangrovi - 0.14 % - 1 hits
Mus musculus - 0.14 % - 1 hits
MAG: Terriglobia - 0.14 % - 1 hits
MAG: Phototrophicaceae - 0.14 % - 1 hits
MAG: Hoeflea - 0.14 % - 1 hits
MAG: Alphaproteobacteria - 0.14 % - 1 hits
MAG: Gammaproteobacteria - 0.14 % - 1 hits
Rubrivivax gelatinosus - 0.14 % - 1 hits


In [ ]:
# Displays the interactable widget.
organism_dropdown.observe(on_dropdown_change, names='value')
num_results_input.observe(on_num_results_change, names='value')
ascending_input.observe(on_ascending_input_change, names='value')

display(organism_dropdown, num_results_input, ascending_input, output_organism)

Dropdown(description='Organism:', options=('Klebsiella pneumoniae', 'Escherichia coli', 'Aspergillus citrinote…

BoundedIntText(value=3, description='Top Grade #:', max=736, min=1)

Checkbox(value=False, description='Ascending Grade Order')

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [ ]:
# Quick and Dirty Identified vs Nonidentified.
print(f"{clean_hits.shape[0] / ( clean_contig_query.shape[0] + clean_noncontig_query.shape[0]) * 100:.2f}% of sequences were identified.")

77.31% of sequences were identified.
